In [59]:
import pandas as pd
import billboard
import datetime
import math

### Getting Track Names and Peak Position

In [60]:
song_name_peak_pos = []

In [61]:
#Get hot-100 every 10 days since 2010
start_date = datetime.date(year=2010,day=1,month=1)

while start_date < datetime.date.today():

    print(start_date)
    
    chart = billboard.ChartData("hot-100",date=str(start_date))
    
    for i in chart:
        song_name_peak_pos.append((i.title,i.peakPos))
    
    start_date += datetime.timedelta(10)

2010-01-01
2010-01-11
2010-01-21
2010-01-31
2010-02-10
2010-02-20
2010-03-02
2010-03-12
2010-03-22
2010-04-01
2010-04-11
2010-04-21
2010-05-01
2010-05-11
2010-05-21
2010-05-31
2010-06-10
2010-06-20
2010-06-30
2010-07-10
2010-07-20
2010-07-30
2010-08-09
2010-08-19
2010-08-29
2010-09-08
2010-09-18
2010-09-28
2010-10-08
2010-10-18
2010-10-28
2010-11-07
2010-11-17
2010-11-27
2010-12-07
2010-12-17
2010-12-27
2011-01-06
2011-01-16
2011-01-26
2011-02-05
2011-02-15
2011-02-25
2011-03-07
2011-03-17
2011-03-27
2011-04-06
2011-04-16
2011-04-26
2011-05-06
2011-05-16
2011-05-26
2011-06-05
2011-06-15
2011-06-25
2011-07-05
2011-07-15
2011-07-25
2011-08-04
2011-08-14
2011-08-24
2011-09-03
2011-09-13
2011-09-23
2011-10-03
2011-10-13
2011-10-23
2011-11-02
2011-11-12
2011-11-22
2011-12-02
2011-12-12
2011-12-22
2012-01-01
2012-01-11
2012-01-21
2012-01-31
2012-02-10
2012-02-20
2012-03-01
2012-03-11
2012-03-21
2012-03-31
2012-04-10
2012-04-20
2012-04-30
2012-05-10
2012-05-20
2012-05-30
2012-06-09
2012-06-19

In [62]:
pd.DataFrame(song_name_peak_pos).to_csv("song_name_peak_pos.csv")

In [63]:
#Dict to store track names and peak position 
chart_final = {}

In [64]:
#Remove duplicates, and keep highest peak 
entry = ""
peak = 0

for i in sorted(set(song_name_peak_pos)):
        
    if i[0] == entry and i[1] > peak:
        chart_final[i[0]] = i[1]
    
    elif i[0] == entry and i[1] <= peak:
        continue
        
    else:
        entry = i[0]
        peak = i[1]

In [65]:
#Turn dict into list of tuples
chart_final = [(chart_final[x],x) for x in chart_final]

In [66]:
#Sort list 
chart_final = sorted(chart_final)

In [73]:
#Seperate Chart Final into top 30% popular and bottom 70% unpopular 
len_pop = math.floor(.3 * len(chart_final))
len_unpop = len(chart_final) - len_pop

print("Pop: ", len_pop, " Unpop: ", len_unpop)

pop = chart_final[:536]
unpop = chart_final[536:]

Pop:  555  Unpop:  1298


In [74]:
#Export datasets with song rank
pd.DataFrame(pop).to_csv("popular_songs_rank.csv")
pd.DataFrame(unpop).to_csv("unpopular_songs_rank.csv")

In [75]:
#Label Sets 1 popular, 0 unpopular
pop_label = [(x[1],1) for x in pop]
unpop_label = [(x[1],0) for x in unpop]

In [76]:
#Export labeled datasets
pd.DataFrame(pop_label).to_csv("popular_songs_label.csv")
pd.DataFrame(unpop_label).to_csv("unpopular_songs_label.csv")

### Getting Track Features

In [77]:
#Spotipy related setup
import spotipy
import spotipy.util as util
scope = 'user-library-read'
username = 'ethantanen'
SPOTIPY_CLIENT_ID='5433d3c112cb41cb99f0f6aeb350cf80'
SPOTIPY_CLIENT_SECRET='158ff273815044f9a3500df7f1c2358c'
SPOTIPY_REDIRECT_URI='https://localhost:8080'
token = util.prompt_for_user_token(username,scope,client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
spotify = spotipy.Spotify(auth=token)

In [78]:
#Read in Files
pop_label = pd.read_csv("popular_songs_label.csv",index_col=0)
unpop_label = pd.read_csv("unpopular_songs_label.csv",index_col=0)

pop_label.columns = ["Track","Label"]
unpop_label.columns = ["Track","Label"]

In [79]:
#Get ids for popular songs
pop_ids = []
pop_tracks = list(pop_label["Track"])
count = 0

for track in pop_label["Track"]:
    print(count)
    search = spotify.search(track,type="track",limit=1)['tracks']['items']
    
    if len(search) > 0:
        pop_ids.append(search[0]['id'])
    else:
        del pop_tracks[count]
    count += 1





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

In [32]:
#Save the table of song names and id's... just in case!
pd.DataFrame(pop_ids).to_csv("pop_ids.csv")

In [49]:
#Get features for popular songs
pop_features = {}

ids = []
trks = []

for i in range(0,6):
    ids.append(pop_ids[i*100:(i+1)*100])
    trks.append(pop_tracks[i*100:(i+1)*100])

for i,t in zip(ids,trks):
    features = spotify.audio_features(i)
    
    for t,f in zip(t,features):
        pop_features[t] = f


In [53]:
#Create dataframe from the audio features 
pop_features = pd.DataFrame(pop_features).transpose()

In [57]:
#Preview of table 
len(pop_features["energy"])

536

In [80]:
print(len(unpop_label["Track"]))

1317


In [81]:
#Get ids for unpopular songs
unpop_ids = []
unpop_tracks = list(unpop_label["Track"])
count = 0

for track in unpop_label["Track"]:
    print(count)
    search = spotify.search(track,type="track",limit=1)['tracks']['items']
    
    if len(search) > 0:
        unpop_ids.append(search[0]['id'])
    else:
        print("HERE")
        del unpop_tracks[count]
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
HERE
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [82]:
#Save the table of song names and id's... just in case!
pd.DataFrame(unpop_ids).to_csv("unpop_ids.csv")

In [83]:
#Get features for popular songs
unpop_features = {}

ids = []
trks = []

for i in range(0,12):
    ids.append(unpop_ids[i*100:(i+1)*100])
    trks.append(unpop_tracks[i*100:(i+1)*100])

for i,t in zip(ids,trks):
    features = spotify.audio_features(i)
    
    for t,f in zip(t,features):
        unpop_features[t] = f

In [84]:
#Create dataframe from the audio features 
unpop_features = pd.DataFrame(unpop_features).transpose()

In [89]:
#Preview of table 
unpop_features.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
'Til Summer Comes Around,0.593,https://api.spotify.com/v1/audio-analysis/1CKm...,0.57,331467,0.629,1CKmI1IQjVEVB3F7VmJmM3,0.000136,9,0.77,-7.608,0,0.0331,127.907,4,https://api.spotify.com/v1/tracks/1CKmI1IQjVEV...,audio_features,spotify:track:1CKmI1IQjVEVB3F7VmJmM3,0.308
(Kissed You) Good Night,0.0807,https://api.spotify.com/v1/audio-analysis/3d1H...,0.523,232903,0.747,3d1Hbu6v1uJ2rTClxKYYRl,0,11,0.11,-6.125,1,0.0292,118.97,4,https://api.spotify.com/v1/tracks/3d1Hbu6v1uJ2...,audio_features,spotify:track:3d1Hbu6v1uJ2rTClxKYYRl,0.294
***Flawless,0.241,https://api.spotify.com/v1/audio-analysis/0zVM...,0.639,234413,0.659,0zVMzJ37VQNFUNvdxxat2E,8.33e-06,8,0.484,-7.273,1,0.215,136.801,4,https://api.spotify.com/v1/tracks/0zVMzJ37VQNF...,audio_features,spotify:track:0zVMzJ37VQNFUNvdxxat2E,0.457
1 Night,0.366,https://api.spotify.com/v1/audio-analysis/40yJ...,0.762,243270,0.476,40yJGQyEaBbIkXpwMFkXip,0,0,0.103,-12.686,1,0.114,130.007,4,https://api.spotify.com/v1/tracks/40yJGQyEaBbI...,audio_features,spotify:track:40yJGQyEaBbIkXpwMFkXip,0.375
10 Bands,0.23,https://api.spotify.com/v1/audio-analysis/12d5...,0.835,177733,0.756,12d5QFwzh60IIHlsSnAvps,0,7,0.0801,-3.789,1,0.382,173,4,https://api.spotify.com/v1/tracks/12d5QFwzh60I...,audio_features,spotify:track:12d5QFwzh60IIHlsSnAvps,0.87


In [90]:
#

for i in unpop_features.acousticness:
    print(i)

0.593
0.0807
0.241
0.366
0.23
0.763
0.0113
0.319
0.0564
0.183
0.0293
0.157
0.124
0.0148
0.122
0.0246
0.000284
0.0665
0.083
0.0158
0.282
0.309
0.245
0.0487
0.219
0.126
0.0543
0.222
0.154
0.151
0.00155
0.0249
0.042
0.00104
0.0124
0.409
0.0275
0.0014
0.0275
0.023
0.000283
0.922
0.0724
0.14
0.32
0.0527
0.00916
0.062
0.0508
0.0312
0.00159
0.798
0.0504
0.0734
0.29
0.162
0.00101
0.0457
0.508
0.0476
0.512
0.0134
0.0151
0.0746
0.207
0.114
0.00767
0.312
0.429
0.00172
0.0418
0.564
0.039
0.46
0.356
0.134
0.0128
0.707
0.00226
0.124
0.0639
0.096
0.00505
0.017
0.0677
0.996
0.017
0.0352
0.06
0.078
0.378
0.177
0.00305
0.192
0.107
0.165
0.0525
0.0145
0.0849
0.208
0.0611
0.0886
0.122
0.344
0.00402
0.0491
0.00898
0.313
0.000232
0.214
0.0424
0.00661
0.0868
0.0223
0.0278
0.0199
0.164
0.00841
0.579
0.228
0.0629
0.000398
0.219
0.143
0.416
0.0571
0.217
0.297
0.139
0.139
0.261
0.0581
0.0348
0.0538
0.13
0.0348
0.181
0.0108
0.0219
0.346
0.723
0.385
0.00373
0.0136
0.00863
0.463
0.0119
0.254
0.117
0.095
0.000831
0.

In [94]:
pd.DataFrame(pop_features).to_csv("pop_features.csv")
pd.DataFrame(unpop_features).to_csv("unpop_features.csv")